In [ ]:
# iot_device.py

import json
import os
import time
import requests
import uuid
import logging
import psutil
import argparse
import random
import threading

class IoTDevice:
    def __init__(self, config_file, device_name):
        self.config = self.load_config(config_file, device_name)
        self.device_id = self.config['device_id']
        self.server_url = self.config['server_url']
        self.pep_url = self.config['pep_url']
        self.data_generation_interval = self.config.get('data_generation_interval', 60)
        self.send_interval = self.config.get('send_interval', 120)
        self.data_type = self.config.get('data_type', 'generic')
        self.min_value = self.config.get('min_value', 0)
        self.max_value = self.config.get('max_value', 100)
        self.last_generated_data = None
        self.running = True
        self.setup_logging()

    def setup_logging(self):
        logging.basicConfig(
            filename=f'{self.device_id}.log',
            level=logging.INFO,
            format='%(asctime)s %(levelname)s: %(message)s'
        )

    def load_config(self, config_file, device_name):
        try:
            with open(config_file, 'r') as f:
                configs = json.load(f)
            config = configs.get(device_name)
            if not config:
                raise ValueError(f"Configuration for device '{device_name}' not found.")
            return config
        except Exception as e:
            print(f"Error loading device configuration: {e}")
            exit(1)

    def start(self):
        # Start threads for data generation and sending
        threading.Thread(target=self.generate_data_loop).start()
        threading.Thread(target=self.send_data_loop).start()

    def generate_data_loop(self):
        while self.running:
            self.generate_data()
            time.sleep(self.data_generation_interval)

    def send_data_loop(self):
        while self.running:
            time.sleep(self.send_interval)
            self.send_data_to_server()

    def generate_data(self):
        # Generate a random number between min_value and max_value
        self.last_generated_data = round(random.uniform(self.min_value, self.max_value), 2)
        logging.info(f"{self.device_id}: Generated data: {self.last_generated_data}")

    def send_data_to_server(self):
        # Simulate sending data to the server via PEP
        resource_id = 'send_data_to_server'
        request_data = {
            'data': self.last_generated_data
        }
        logging.info(f"{self.device_id}: Attempting to send data to server.")

        # Use PEP to handle the request over the network
        self.handle_request(resource_id, request_data)

    def handle_user_request(self):
        # Simulate handling a user request to read data via PEP
        resource_id = f'{self.device_id}_data'
        request_data = {}  # No additional data needed
        logging.info(f"{self.device_id}: Received user request to read data.")

        # Use PEP to handle the request over the network
        self.handle_request(resource_id, request_data)

    def handle_request(self, resource_id, request_data):
        """
        Sends a request to the PEP service and handles the response.
        """
        # Generate a unique request ID
        request_id = str(uuid.uuid4())

        # Start CPU and timing measurements
        process = psutil.Process(os.getpid())
        cpu_times_start = process.cpu_times()
        start_time = time.time()

        # Prepare the request payload
        payload = {
            'entity_id': self.device_id,
            'resource_id': resource_id,
            'request_data': request_data,
            'request_id': request_id
        }

        try:
            # Send a POST request to the PEP service
            pep_response = requests.post(f'{self.pep_url}/handle_request', json=payload)
            pep_response.raise_for_status()

            response_data = pep_response.json()
            access_decision = response_data.get('access_decision')
            logging.info(f"{self.device_id}: Access decision from PEP: {access_decision}")

            if access_decision == 'allow':
                self.perform_action(resource_id, request_data)
            else:
                logging.warning(f"{self.device_id}: Access denied by PEP for resource '{resource_id}'")
        except requests.exceptions.RequestException as e:
            logging.error(f"{self.device_id}: Error communicating with PEP: {e}")
        except Exception as e:
            logging.error(f"{self.device_id}: Unexpected error: {e}")

        # End CPU and timing measurements
        end_time = time.time()
        cpu_times_end = process.cpu_times()

        # Calculate CPU time and processing time
        user_cpu_time = cpu_times_end.user - cpu_times_start.user
        system_cpu_time = cpu_times_end.system - cpu_times_start.system
        total_cpu_time = user_cpu_time + system_cpu_time
        processing_time = end_time - start_time

        # Log the performance data
        logging.info(
            f"Request ID: {request_id} | Resource ID: {resource_id} | Processing Time: {processing_time:.6f}s | "
            f"CPU Time: User={user_cpu_time:.6f}s System={system_cpu_time:.6f}s Total={total_cpu_time:.6f}s"
        )

    def perform_action(self, resource_id, request_data):
        """
        Performs the action after receiving 'allow' from PEP.
        For example, sends data to the server or responds to user requests.
        """
        if resource_id == 'send_data_to_server':
            # Simulate sending data to the server
            logging.info(f"{self.device_id}: Data sent to server: {self.last_generated_data}")
            # Optionally, send data directly to the server if needed
            # self.send_data_directly_to_server()
        elif resource_id.endswith('_data'):
            # Simulate sending data to the user
            logging.info(f"{self.device_id}: Data sent to user: {self.last_generated_data}")
            # Implement actual data sending logic if needed
        else:
            logging.warning(f"{self.device_id}: Unknown resource '{resource_id}'")

    def stop(self):
        self.running = False

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='IoT Device Simulator')
    parser.add_argument('--device_name', type=str, required=True, help='Name of the device')
    parser.add_argument('--config_file', type=str, default='device_config.json', help='Configuration file')

    args = parser.parse_args()

    device = IoTDevice(args.config_file, args.device_name)
    device.start()